In [6]:
import os

In [7]:
%pwd

'/Users/nikhilvinay/Documents/my_projects/end2end-ml-mlflow-wine-quality-prediction/research'

In [8]:
os.chdir('../')

In [9]:
%pwd

'/Users/nikhilvinay/Documents/my_projects/end2end-ml-mlflow-wine-quality-prediction'

In [ ]:
import pandas as pd
import mlflow
import dagshub


In [22]:
dagshub.init(repo_owner='nvinay1303', repo_name='end2end-ml-mlflow-wine-quality-prediction', mlflow=True)
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

/Users/nikhilvinay/Documents/my_projects/end2end-ml-mlflow-wine-quality-prediction/.env/lib/python3.13/site-package
s/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=5ed490b3-a0d0-4bf1-9dc2-b802c9aeb3b3&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=6291efe8cb95010c74a1104842de1665b1ac436870e3dda5bffd3d93ec4028f8


[2025-04-02 00:26:04,243: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"]


[2025-04-02 00:26:04,910: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/access_token "HTTP/1.1 200 OK"]
[2025-04-02 00:26:05,154: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as nvinay1303

[2025-04-02 00:26:05,162: INFO: helpers: Accessing as nvinay1303]
[2025-04-02 00:26:05,588: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/nvinay1303/end2end-ml-mlflow-wine-quality-prediction "HTTP/1.1 200 OK"]
[2025-04-02 00:26:05,826: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "nvinay1303/end2end-ml-mlflow-wine-quality-prediction"

[2025-04-02 00:26:05,828: INFO: helpers: Initialized MLflow to track repo "nvinay1303/end2end-ml-mlflow-wine-quality-prediction"]


Repository nvinay1303/end2end-ml-mlflow-wine-quality-prediction initialized!

[2025-04-02 00:26:05,829: INFO: helpers: Repository nvinay1303/end2end-ml-mlflow-wine-quality-prediction initialized!]
🏃 View run placid-skunk-717 at: https://dagshub.com/nvinay1303/end2end-ml-mlflow-wine-quality-prediction.mlflow/#/experiments/0/runs/1f9fc01167f4400badaac658747e93a3
🧪 View experiment at: https://dagshub.com/nvinay1303/end2end-ml-mlflow-wine-quality-prediction.mlflow/#/experiments/0


In [23]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [24]:
from src.ml_project.constants import *
from src.ml_project.utils.common import read_yaml, create_directories, save_json

In [25]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/nvinay1303/end2end-ml-mlflow-wine-quality-prediction.mlflow",
           
        )

        return model_evaluation_config


In [26]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [27]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [28]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Create an input example for MLflow
        input_example = test_x.iloc[:1]  # Select first row as an example

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # Save metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(
                    model,
                    "model",
                    registered_model_name="ElasticnetModel",
                    input_example=input_example  # ✅ Fix: Add input example
                )
            else:
                mlflow.sklearn.log_model(
                    model,
                    "model",
                    input_example=input_example  # ✅ Fix: Add input example
                )

In [30]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-04-02 00:27:47,490: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-04-02 00:27:47,492: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-02 00:27:47,493: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-02 00:27:47,494: INFO: common: created directory at: artifacts]
[2025-04-02 00:27:47,494: INFO: common: created directory at: artifacts/model_evaluation]
[2025-04-02 00:27:47,727: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/04/02 00:27:56 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.


🏃 View run abrasive-ape-925 at: https://dagshub.com/nvinay1303/end2end-ml-mlflow-wine-quality-prediction.mlflow/#/experiments/0/runs/852c996140b546a3b36ff6a3b6ce1c0d
🧪 View experiment at: https://dagshub.com/nvinay1303/end2end-ml-mlflow-wine-quality-prediction.mlflow/#/experiments/0
